In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
os.chdir('/content/drive/My Drive/ML/Major')
os.listdir()




['FIFA Player Database',
 'Bundesliga.csv',
 'Bundesliga_skysports.ipynb',
 'Premier_League.csv',
 'Premier_League (1).gsheet',
 'Premier_League.gsheet',
 'fifa_player_rating_scrape.py']

In [ ]:
df = pd.read_csv('Bundesliga.csv',index_col=0)

In [ ]:
df.head()

,year,home_team,home_goals,home_Possession %,home_Total Shots,home_On Target,home_Off Target,home_Blocked,home_Passing %,home_Clear-Cut Chances,home_Corners,home_Offsides,home_Tackles %,home_Aerial Duels %,home_Saves,home_Fouls Committed,home_Fouls Won,home_Yellow Cards,home_Red Cards,away_team,away_goals,away_Possession %,away_Total Shots,away_On Target,away_Off Target,away_Blocked,away_Passing %,away_Clear-Cut Chances,away_Corners,away_Offsides,away_Tackles %,away_Aerial Duels %,away_Saves,away_Fouls Committed,away_Fouls Won,away_Yellow Cards,away_Red Cards,home_form_1,home_form_2,home_form_3,home_form_4,home_form_5,home_form_6,away_form_1,away_form_2,away_form_3,away_form_4,away_form_5,away_form_6,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11
0,2014-15,Bay Munich,2,74.2,20,7,9,4,87.6,1,5,1,83.3,48.8,5,11,10,0,0,Mainz,0,25.8,7,5,2,0,65.6,0,3,2,85.7,51.2,5,11,11,0,0,6 - 2,2 - 1,2 - 1,5 - 0,4 - 0,4 - 1,1 - 3,2 - 3,0 - 1,0 - 4,1 - 4,1 - 3,M Neuer,Rafinha,J Boateng,Dante,J Bernat,P Lahm,X Alonso,B Schweinsteiger,T Müller,R Lewandowski,M Götze,L Karius,S Bell,N Bungert,N Noveski,P Bengtsson,D Brosinski,J Geis,J Baumgartlinger,Jairo,P De Blasis,Y Malli
1,2014-15,B Dortmund,3,66.3,23,8,9,6,79.8,5,9,0,69.6,63.6,3,10,3,0,0,W Bremen,2,33.7,8,5,3,0,65.8,2,1,3,92.3,36.4,5,3,10,0,0,3 - 0,2 - 0,2 - 0,3 - 0,1 - 3,1 - 0,1 - 1,1 - 1,1 - 1,1 - 0,3 - 1,1 - 4,R Weidenfeller,E Durm,N Subotic,M Hummels,M Schmelzer,I Gündogan,S Kehl,H Mkhitaryan,S Kagawa,M Reus,P Aubameyang,K Casteels,T Gebre Selassie,A Lukimya,J Vestergaard,J Sternberg,F Kroos,M Busch,F Bartels,Z Junuzovic,L Öztunali,F Di Santo
2,2014-15,Cologne,2,36.1,12,3,7,2,76.9,1,4,1,83.3,42.5,3,14,8,0,0,Wolfsburg,2,63.9,15,5,8,2,87.8,2,8,2,92.3,57.5,2,11,14,0,0,1 - 2,1 - 1,1 - 1,1 - 3,0 - 1,2 - 3,1 - 1,2 - 1,1 - 1,0 - 0,1 - 2,1 - 1,T Horn,M Brecko,D Maroh,K Wimmer,D Svento,M Risse,M Lehmann,Y Gerhardt,K Nagasawa,Y Osako,A Ujah,M Grün,Vieirinha,R Knoche,T Klose,M Schäfer,M Arnold,Luiz Gustavo,I Perisic,K De Bruyne,D Caligiuri,B Dost
3,2014-15,Frankfurt,2,41.6,8,3,3,2,66.8,1,3,4,77.3,52.0,6,20,13,2,0,Leverkusen,1,58.4,18,7,7,4,79.8,0,8,3,87.5,48.0,2,14,20,0,0,1 - 1,0 - 5,1 - 1,2 - 1,3 - 1,1 - 1,4 - 2,0 - 1,3 - 1,6 - 2,1 - 0,1 - 1,K Trapp,T Chandler,C Zambrano,A Madlung,B Oczipka,M Russ,M Hasebe,S Aigner,M Stendera,L Waldschmidt,H Seferovic,B Leno,S Boenisch,T Jedvaj,L Bender,B Wendell,S Reinartz,H Calhanoglu,J Brandt,K Bellarabi,H Son,S Kießling
4,2014-15,Hamburg,2,53.9,10,4,4,2,74.5,2,3,5,75.0,62.3,3,15,18,1,0,Schalke,0,46.1,12,4,5,3,70.8,0,6,2,78.9,37.7,2,18,14,2,0,2 - 2,3 - 1,3 - 0,1 - 0,4 - 3,2 - 1,4 - 1,1 - 1,0 - 3,1 - 1,0 - 4,1 - 3,R Adler,H Westermann,J Djourou,S Rajkovic,M Ostrzolek,I Olic,G Kacar,M Díaz,I Ilicevic,L Holtby,P Lasogga,R Fährmann,M Friedrich,J Matip,M Nastasic,S Kolasinac,R Neustädter,J Farfán,M Höger,M Meyer,L Sané,K Huntelaar


In [ ]:
df.year = df.year.astype('str')

In [ ]:
df.columns

Index(['year', 'home_team', 'home_goals', 'home_Possession %',
       'home_Total Shots', 'home_On Target', 'home_Off Target', 'home_Blocked',
       'home_Passing %', 'home_Clear-Cut Chances', 'home_Corners',
       'home_Offsides', 'home_Tackles %', 'home_Aerial Duels %', 'home_Saves',
       'home_Fouls Committed', 'home_Fouls Won', 'home_Yellow Cards',
       'home_Red Cards', 'away_team', 'away_goals', 'away_Possession %',
       'away_Total Shots', 'away_On Target', 'away_Off Target', 'away_Blocked',
       'away_Passing %', 'away_Clear-Cut Chances', 'away_Corners',
       'away_Offsides', 'away_Tackles %', 'away_Aerial Duels %', 'away_Saves',
       'away_Fouls Committed', 'away_Fouls Won', 'away_Yellow Cards',
       'away_Red Cards', 'home_form_1', 'home_form_2', 'home_form_3',
       'home_form_4', 'home_form_5', 'home_form_6', 'away_form_1',
       'away_form_2', 'away_form_3', 'away_form_4', 'away_form_5',
       'away_form_6', 'home_player_1', 'home_player_2', 'home_player

In [ ]:
home_cols = ['home_Possession %',
       'home_Total Shots', 'home_On Target', 'home_Off Target', 'home_Blocked',
       'home_Passing %', 'home_Clear-Cut Chances', 'home_Corners',
       'home_Offsides', 'home_Tackles %', 'home_Aerial Duels %', 'home_Saves',
       'home_Fouls Committed', 'home_Fouls Won', 'home_Yellow Cards',
       'home_Red Cards']
away_cols = ['away_Possession %',
       'away_Total Shots', 'away_On Target', 'away_Off Target', 'away_Blocked',
       'away_Passing %', 'away_Clear-Cut Chances', 'away_Corners',
       'away_Offsides', 'away_Tackles %', 'away_Aerial Duels %', 'away_Saves',
       'away_Fouls Committed', 'away_Fouls Won', 'away_Yellow Cards',
       'away_Red Cards']

In [ ]:
home_form = ['home_form_1', 'home_form_2', 'home_form_3',
       'home_form_4', 'home_form_5', 'home_form_6']

away_form = ['away_form_1',
       'away_form_2', 'away_form_3', 'away_form_4', 'away_form_5',
       'away_form_6']

In [ ]:
df['home_goal_difference'] = df.home_goals - df.away_goals
df['away_goal_difference'] = df.away_goals - df.home_goals

In [ ]:
teams = df.home_team.unique()


In [ ]:
home_cols.append('home_goals')
home_cols.append('home_goal_difference')
away_cols.append('away_goals')
away_cols.append('away_goal_difference')

for team in teams:
  idxs = df[df.home_team==team].index
  for col in home_cols:
    # a = df.loc[idxs,col].rolling(window=5).mean()
    # b = [np.nan]
    # b[1:] = a
    df.loc[idxs,col+'_moving'] = df.loc[idxs,col].rolling(window=5).mean().shift()
  idxs = df[df.away_team==team].index
  for col in away_cols:
    # a = df.loc[idxs,col].rolling(window=5).mean()
    # b = [np.nan]
    # b[1:] = a
    df.loc[idxs,col+'_moving'] = df.loc[idxs,col].rolling(window=5).mean().shift()

home_cols.remove('home_goals')
home_cols.remove('home_goal_difference')
away_cols.remove('away_goals')
away_cols.remove('away_goal_difference')

In [ ]:
data = df.copy()
df_new = df.copy()
df_new = df_new[df_new.year>'2014-15']

In [ ]:
years = ['2015-16', '2016-17', '2017-18', '2018-19', '2019-20']
for year in years:
  df = data[data.year<year]
  home_dfs = {}
  away_dfs = {}
  for team in teams:
    avg_home_stats = df[df.home_team==team][home_cols].mean()
    avg_away_stats = df[df.away_team==team][away_cols].mean()
    
    home_dfs[team] = 0.7*avg_home_stats.values + 0.3*avg_away_stats.values
    away_dfs[team] = 0.7*avg_away_stats.values + 0.3*avg_home_stats.values

    idxs = df_new[(df_new.home_team==team) & (df_new.year==year)].index
    df_new.loc[idxs,home_cols] = home_dfs[team]

    idxs = df_new[(df_new.away_team==team) & (df_new.year==year)].index
    df_new.loc[idxs,away_cols] = away_dfs[team]

In [ ]:
df_new['home_player_2']

198     R Hilbert
199        P Lahm
200      S Bender
201       D Opare
202       F Schär
          ...    
1195     N Elvedi
1196     N Elvedi
1197    J Kimmich
1198     N Elvedi
1199       M Lang
Name: home_player_2, Length: 1000, dtype: object

In [ ]:
df_new[df_new.home_team==teams[0]]

,year,home_team,home_goals,home_Possession %,home_Total Shots,home_On Target,home_Off Target,home_Blocked,home_Passing %,home_Clear-Cut Chances,home_Corners,home_Offsides,home_Tackles %,home_Aerial Duels %,home_Saves,home_Fouls Committed,home_Fouls Won,home_Yellow Cards,home_Red Cards,away_team,away_goals,away_Possession %,away_Total Shots,away_On Target,away_Off Target,away_Blocked,away_Passing %,away_Clear-Cut Chances,away_Corners,away_Offsides,away_Tackles %,away_Aerial Duels %,away_Saves,away_Fouls Committed,away_Fouls Won,away_Yellow Cards,away_Red Cards,home_form_1,home_form_2,home_form_3,...,away_player_10,away_player_11,home_goal_difference,away_goal_difference,home_Possession %_moving,home_Total Shots_moving,home_On Target_moving,home_Off Target_moving,home_Blocked_moving,home_Passing %_moving,home_Clear-Cut Chances_moving,home_Corners_moving,home_Offsides_moving,home_Tackles %_moving,home_Aerial Duels %_moving,home_Saves_moving,home_Fouls Committed_moving,home_Fouls Won_moving,home_Yellow Cards_moving,home_Red Cards_moving,home_goals_moving,home_goal_difference_moving,away_Possession %_moving,away_Total Shots_moving,away_On Target_moving,away_Off Target_moving,away_Blocked_moving,away_Passing %_moving,away_Clear-Cut Chances_moving,away_Corners_moving,away_Offsides_moving,away_Tackles %_moving,away_Aerial Duels %_moving,away_Saves_moving,away_Fouls Committed_moving,away_Fouls Won_moving,away_Yellow Cards_moving,away_Red Cards_moving,away_goals_moving,away_goal_difference_moving
199,2015-16,Bay Munich,3,71.039500,17.420000,7.065000,6.945000,3.410000,87.849500,1.250000,5.795000,2.370000,78.047500,52.167500,2.210000,11.760000,14.170000,0.965000,0.140000,Hannover,1,52.360909,11.772727,4.563636,4.554545,2.654545,74.009091,1.181818,6.027273,2.509091,79.995455,54.004545,3.300000,18.290909,14.700000,2.400000,0.181818,6 - 2,2 - 1,2 - 1,...,F Klaus,A Sobiech,2,-2,71.46,19.8,8.2,8.0,3.6,87.44,1.2,8.0,3.4,78.84,58.18,1.8,7.4,14.8,0.8,0.2,3.2,2.8,61.12,15.0,5.6,6.2,3.2,78.28,1.4,7.0,1.6,78.84,54.94,3.0,18.6,15.4,2.6,0.0,1.8,-0.6
216,2015-16,Bay Munich,1,71.039500,17.420000,7.065000,6.945000,3.410000,87.849500,1.250000,5.795000,2.370000,78.047500,52.167500,2.210000,11.760000,14.170000,0.965000,0.140000,M'gladbach,1,55.744833,11.760000,4.635000,4.533333,2.591667,82.335167,1.941667,4.708333,1.793333,79.294000,49.844667,3.780000,11.536667,13.470000,1.763333,0.030000,6 - 2,2 - 1,2 - 1,...,T Hazard,Raffael,0,0,69.04,19.0,8.6,7.2,3.2,86.40,1.6,7.4,4.8,75.38,55.60,2.0,8.4,15.2,0.8,0.2,3.0,2.6,56.52,12.6,3.2,5.2,4.2,81.48,1.4,6.4,1.2,74.80,50.50,4.4,12.2,13.4,1.4,0.0,1.0,0.2
237,2015-16,Bay Munich,3,71.039500,17.420000,7.065000,6.945000,3.410000,87.849500,1.250000,5.795000,2.370000,78.047500,52.167500,2.210000,11.760000,14.170000,0.965000,0.140000,Schalke,0,51.645455,12.627273,4.318182,5.272727,3.036364,76.379091,0.754545,4.990909,2.572727,72.779091,53.597273,3.772727,15.945455,14.290909,1.736364,0.027273,6 - 2,2 - 1,2 - 1,...,K Huntelaar,M Choupo-Moting,3,-3,65.64,15.6,6.4,6.8,2.4,84.28,1.4,8.2,3.6,76.58,49.18,1.8,9.8,14.8,1.2,0.2,1.6,1.0,49.04,11.8,6.8,3.0,2.0,78.90,1.6,3.8,3.8,69.32,48.98,2.6,16.4,13.0,1.0,0.0,2.8,2.0
250,2015-16,Bay Munich,1,71.039500,17.420000,7.065000,6.945000,3.410000,87.849500,1.250000,5.795000,2.370000,78.047500,52.167500,2.210000,11.760000,14.170000,0.965000,0.140000,Frankfurt,0,52.981818,11.690909,4.209091,4.481818,3.000000,71.982727,1.263636,4.290909,3.054545,79.275455,49.872727,3.509091,17.227273,16.845455,2.672727,0.063636,6 - 2,2 - 1,2 - 1,...,A Ben-Hatira,H Seferovic,1,-1,67.04,16.4,7.4,6.0,3.0,85.00,1.6,8.4,3.2,80.16,47.36,1.8,9.2,12.2,0.6,0.0,2.0,1.6,46.66,10.6,4.8,3.4,2.4,63.20,1.2,4.2,3.4,78.78,41.26,3.2,16.8,14.6,2.2,0.0,1.0,-0.8
271,2015-16,Bay Munich,5,71.039500,17.420000,7.065000,6.945000,3.410000,87.849500,1.250000,5.795000,2.370000,78.047500,52.167500,2.210000,11.760000,14.170000,0.965000,0.140000,W Bremen,0,43.270455,12.290909,5.109091,4.895455,2.286364,66.475000,1.072727,3.686364,2.000000,79.389

In [ ]:
df_new.drop(columns=home_form,inplace=True)
df_new.drop(columns=away_form,inplace=True)

In [ ]:
df_new.loc[idxs,home_cols]

,home_Possession %,home_Total Shots,home_On Target,home_Off Target,home_Blocked,home_Passing %,home_Clear-Cut Chances,home_Corners,home_Offsides,home_Tackles %,home_Aerial Duels %,home_Saves,home_Fouls Committed,home_Fouls Won,home_Yellow Cards,home_Red Cards
1003,51.925049,15.067792,5.472240,5.986006,3.609545,78.791633,1.661786,4.618929,2.308117,72.673753,50.510295,3.507695,14.362208,14.265227,1.868701,0.032143
1019,44.250543,11.597826,3.962352,5.167984,2.467490,73.670692,1.467490,4.574012,2.539921,73.396581,49.171255,3.428063,13.449704,13.789723,1.740810,0.074506
1037,53.730731,13.853021,5.133821,5.313255,3.405945,81.413060,1.821053,4.911404,2.373489,71.763830,50.889074,3.243372,11.498928,12.660331,1.341813,0.031189
1062,47.543114,10.159091,3.664545,4.023409,2.471136,75.306409,1.156136,3.855682,2.125682,72.839455,50.354636,2.950455,14.663182,13.791591,1.753182,0.095227
1077,46.177080,12.048062,4.491835,4.566408,2.989819,73.545674,0.977468,4.445736,1.719897,70.967090,45.843468,3.651266,12.867545,13.010181,1.547752,0.067597
1090,49.808254,12.156924,4.281036,5.045699,2.830189,72.943646,1.671154,5.119060,2.691078,74.844311,48.312098,3.192453,15.199296,14.201247,2.386633,0.087624
1112,46.917409,13.390194,5.269768,4.958051,3.162375,72.724461,1.530294,4.851159,1.858897,72.026964,50.098669,3.125846,14.975439,14.170457,1.860213,0.040633
1118,59.152639,15.331944,5.856944,5.818056,3.656944,81.439583,2.483333,5.509722,2.666667,72.045556,51.998056,2.204167,11.670833,12.494444,1.336111,0.043056
1141,53.132581,14.206831,5.362334,5.816509,3.027989,76.352495,1.908159,5.478273,2.878937,69.551622,53.715797,2.611480,13.792505,13.301233,1.508349,0.079886
1143,43.750000,12.563636,4.618182,4.881818,3.063636,77.434545,1.627273,4.481818,1.354545,61.456364,49.484545,3.309091,12.272727,11.318182,1.827273,0.027273


In [ ]:
print(df_new.shape)

df_new.describe()

print(df_new.home_player_2)


(1000, 97)
198     R Hilbert
199        P Lahm
200      S Bender
201       D Opare
202       F Schär
          ...    
1195     N Elvedi
1196     N Elvedi
1197    J Kimmich
1198     N Elvedi
1199       M Lang
Name: home_player_2, Length: 1000, dtype: object


In [ ]:
fifa_df = pd.read_csv(r"FIFA Player Database/players_17.csv")


In [ ]:
print(fifa_df)

print(fifa_df.shape)

print(fifa_df.columns)

       sofifa_id                                         player_url  ...   rcb    rb
0          20801  https://sofifa.com/player/20801/c-ronaldo-dos-...  ...  53+4  61+4
1         158023  https://sofifa.com/player/158023/lionel-messi/...  ...  45+4  57+4
2         190871  https://sofifa.com/player/190871/neymar-da-sil...  ...  46+4  58+4
3         167495  https://sofifa.com/player/167495/manuel-neuer/...  ...  33+4  34+4
4         176580  https://sofifa.com/player/176580/luis-suarez/1...  ...  58+4  64+4
...          ...                                                ...  ...   ...   ...
17592     212270  https://sofifa.com/player/212270/mark-foden/17...  ...  22+1  22+1
17593     223621  https://sofifa.com/player/223621/dean-mahon/17...  ...  24+1  23+1
17594     214725  https://sofifa.com/player/214725/sam-ramsbotto...  ...  19+1  19+1
17595     228581  https://sofifa.com/player/228581/ben-kelly/170002  ...  26+1  25+1
17596      11728  https://sofifa.com/player/11728/barry-richards.

In [ ]:
fifa_df_extract = fifa_df[["short_name", "long_name", "overall", "club_name"]]

print(fifa_df_extract)

              short_name  ...          club_name
0      Cristiano Ronaldo  ...        Real Madrid
1               L. Messi  ...       FC Barcelona
2                 Neymar  ...       FC Barcelona
3               M. Neuer  ...  FC Bayern München
4              L. Suárez  ...       FC Barcelona
...                  ...  ...                ...
17592           M. Foden  ...     Ross County FC
17593           D. Mahon  ...        Bohemian FC
17594      S. Ramsbottom  ...      Galway United
17595           B. Kelly  ...            Dundalk
17596      B. Richardson  ...  Wycombe Wanderers

[17597 rows x 4 columns]


In [ ]:
print(df_new.columns)

print(df_new.year.unique())

print(df_new.home_team.unique())

Index(['year', 'home_team', 'home_goals', 'home_Possession %',
       'home_Total Shots', 'home_On Target', 'home_Off Target', 'home_Blocked',
       'home_Passing %', 'home_Clear-Cut Chances', 'home_Corners',
       'home_Offsides', 'home_Tackles %', 'home_Aerial Duels %', 'home_Saves',
       'home_Fouls Committed', 'home_Fouls Won', 'home_Yellow Cards',
       'home_Red Cards', 'away_team', 'away_goals', 'away_Possession %',
       'away_Total Shots', 'away_On Target', 'away_Off Target', 'away_Blocked',
       'away_Passing %', 'away_Clear-Cut Chances', 'away_Corners',
       'away_Offsides', 'away_Tackles %', 'away_Aerial Duels %', 'away_Saves',
       'away_Fouls Committed', 'away_Fouls Won', 'away_Yellow Cards',
       'away_Red Cards', 'home_player_1', 'home_player_2', 'home_player_3',
       'home_player_4', 'home_player_5', 'home_player_6', 'home_player_7',
       'home_player_8', 'home_player_9', 'home_player_10', 'home_player_11',
       'away_player_1', 'away_player_2', 'aw

In [ ]:
player_columns = ['home_player_1', 'home_player_2', 'home_player_3',
       'home_player_4', 'home_player_5', 'home_player_6', 'home_player_7',
       'home_player_8', 'home_player_9', 'home_player_10', 'home_player_11',
       'away_player_1', 'away_player_2', 'away_player_3', 'away_player_4',
       'away_player_5', 'away_player_6', 'away_player_7', 'away_player_8',
       'away_player_9', 'away_player_10', 'away_player_11']


print(player_columns)

players_list = []

for player in player_columns:
  players_list.extend(df[player].tolist())

print(players_list)
print(len(players_list))


rating_dict = {}



['home_player_1', 'home_player_2', 'home_player_3', 'home_player_4', 'home_player_5', 'home_player_6', 'home_player_7', 'home_player_8', 'home_player_9', 'home_player_10', 'home_player_11', 'away_player_1', 'away_player_2', 'away_player_3', 'away_player_4', 'away_player_5', 'away_player_6', 'away_player_7', 'away_player_8', 'away_player_9', 'away_player_10', 'away_player_11']
['M Neuer', 'R Weidenfeller', 'T Horn', 'K Trapp', 'R Adler', 'R Zieler', 'O Baumann', 'Y Sommer', 'L Kruse', 'B Leno', 'M Hitz', 'T Kraft', 'L Karius', 'R Bürki', 'R Fährmann', 'S Ulreich', 'K Casteels', 'D Benaglio', 'L Kruse', 'T Horn', 'P Reina', 'M Langerak', 'K Trapp', 'R Zieler', 'Y Sommer', 'S Ulreich', 'R Adler', 'L Karius', 'T Kraft', 'M Hitz', 'O Baumann', 'R Bürki', 'R Fährmann', 'K Casteels', 'D Benaglio', 'B Leno', 'L Kruse', 'Y Sommer', 'M Langerak', 'T Horn', 'R Adler', 'R Zieler', 'S Ulreich', 'M Neuer', 'L Karius', 'K Casteels', 'D Benaglio', 'B Leno', 'R Weidenfeller', 'T Kraft', 'O Baumann', 'R